In [1]:
# set workspace
WhoIsWorking = "RT"
WorkLocation = "School"

if WhoIsWorking == "RT":
    if WorkLocation == "Home":
        folder_path = r'C:\Users\rentr\Desktop\ENGO\DataMining\ENGO645_TermProject_Data'
    elif WorkLocation == "School":
        folder_path = r'D:\ENGO645_TermProject_Data'
elif WhoIsWorking == "AA":
    folder_path = ''
elif WhoIsWorking == "IA":
    folder_path = r'C:\Users\afoam\assignments\ENGO645_TorontoCrimeAnalysis'
elif WhoIsWorking == "AI":
    folder_path = r'C:\Adewale_Directory\Capacity_Building\MGIS\Lecture_Note\WINTER2024\ENGO645\Course_Project\Project_Data'

print(folder_path)

D:\ENGO645_TermProject_Data


In [2]:
import os

# List all files in the folder
files = os.listdir(folder_path)

# Print the list of files
for file in files:
    print(file)

Archive
AverageHomePrices_2023.csv
BuildingOutlines.CPG
BuildingOutlines.dbf
BuildingOutlines.prj
BuildingOutlines.sbn
BuildingOutlines.sbx
BuildingOutlines.shp
BuildingOutlines.shp.xml
BuildingOutlines.shx
BuildingOutlines_Dissolved.dbf
BuildingOutlines_Dissolved.prj
BuildingOutlines_Dissolved.sbn
BuildingOutlines_Dissolved.sbx
BuildingOutlines_Dissolved.shp
BuildingOutlines_Dissolved.shp.xml
BuildingOutlines_Dissolved.shx
CensusProfile_2021_PopAndIncome_Reformatted.csv
ClimateData_Hourly.csv
EnhancedPointsOfInterestPoint_Toronto.cpg
EnhancedPointsOfInterestPoint_Toronto.dbf
EnhancedPointsOfInterestPoint_Toronto.prj
EnhancedPointsOfInterestPoint_Toronto.sbn
EnhancedPointsOfInterestPoint_Toronto.sbx
EnhancedPointsOfInterestPoint_Toronto.shp
EnhancedPointsOfInterestPoint_Toronto.shp.xml
EnhancedPointsOfInterestPoint_Toronto.shx
LandUseClassified_20211220.cpg
LandUseClassified_20211220.dbf
LandUseClassified_20211220.prj
LandUseClassified_20211220.sbn
LandUseClassified_20211220.sbx
LandUs

In [3]:
import os
import geopandas as gpd
import pandas as pd

# Path to the shapefile for neighborhoods
neighborhoods_shapefile_path = os.path.join(folder_path, 'Neighbourhoods.shp')

# Read the shapefile into a GeoDataFrame
neighbourhoods_gdf = gpd.read_file(neighborhoods_shapefile_path)

# Print the first few lines of the  table
neighbourhoods_gdf.head(2)

,_id1,AREA_ID2,AREA_AT3,PARENT_4,AREA_SH5,AREA_LO6,AREA_NA7,AREA_DE8,CLASSIF9,CLASSIF10,OBJECTI11,geometry
0,1,2502366,26022881,0,174,174,South Eglinton-Davisville,South Eglinton-Davisville (174),Not an NIA or Emerging Neighbourhood,NA,17824737.0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697..."
1,2,2502365,26022880,0,173,173,North Toronto,North Toronto (173),Not an NIA or Emerging Neighbourhood,NA,17824753.0,"POLYGON ((-79.39744 43.70693, -79.39837 43.706..."


In [4]:
# Keep only certain fields and rename fields
neighbourhoods_gdf = neighbourhoods_gdf[['geometry', '_id1', 'AREA_NA7']].rename(columns={'_id1': 'UID', 'AREA_NA7': 'NeighbourhoodName'})

# Print the first few lines of the updated GeoDataFrame
neighbourhoods_gdf.head(2)

,geometry,UID,NeighbourhoodName
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto


In [5]:
# Path to the shapefile for poles
poles_shapefile_path = os.path.join(folder_path, 'Poles.shp')

# Read shapefiles into GeoDataFrames
poles_gdf = gpd.read_file(poles_shapefile_path)

# List fields in the poles GeoDataFrame
poles_fields = poles_gdf.columns
print("Fields in the poles GeoDataFrame:")
print(poles_fields)

Fields in the poles GeoDataFrame:
Index(['objectid', 'subtype', 'subtypecod', 'elevation', 'X', 'Y', 'geometry'], dtype='object')


In [6]:
# List unique attributes in the 'subtype' field of poles
unique_subtypes = poles_gdf['subtype'].unique()

# Print unique attributes
print("Unique attributes in the 'subtype' field of poles:")
print(unique_subtypes)

Unique attributes in the 'subtype' field of poles:
['Miscellaneous Pole' 'Street Light and Traffic Signal Pole'
 'Traffic Signal Pole' 'Street Light Pole' 'Pedestrian Light Pole'
 'Circular Hydro Tower']


In [7]:
# Filter poles to only keep certain subtypes
desired_subtypes = ['Street Light and Traffic Signal Pole', 'Street Light Pole', 'Pedestrian Light Pole']
filtered_poles_gdf = poles_gdf[poles_gdf['subtype'].isin(desired_subtypes)]

In [8]:
# Perform spatial join to count the number of poles in each neighborhood
poles_per_neighborhood = gpd.sjoin(neighbourhoods_gdf, filtered_poles_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of poles as a new column to the neighborhoods GeoDataFrame
neighbourhoods_gdf['Poles_Count'] = neighbourhoods_gdf['NeighbourhoodName'].map(poles_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head(2)

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geometry,UID,NeighbourhoodName,Poles_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286


In [9]:
# Path to the shapefile for POI
poi_shapefile_path = os.path.join(folder_path, 'EnhancedPointsOfInterestPoint_Toronto.shp')

# Read the shapefile into a GeoDataFrame
poi_gdf = gpd.read_file(poi_shapefile_path)

# Perform spatial join to count the number of POIs within each neighbourhood
poi_per_neighborhood = gpd.sjoin(neighbourhoods_gdf, poi_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of POIs as a new column to the neighborhoods GeoDataFrame
neighbourhoods_gdf['POI_Count'] = neighbourhoods_gdf['NeighbourhoodName'].map(poi_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head()

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281


In [10]:
# Path to the shapefile for traffic cameras
traffic_cameras_shapefile_path = os.path.join(folder_path, 'TrafficCamera.shp')

# Read the shapefile into a GeoDataFrame
traffic_cameras_gdf = gpd.read_file(traffic_cameras_shapefile_path)

# Perform spatial join to count the number of traffic cameras within each neighborhood
traffic_cameras_per_neighborhood = gpd.sjoin(neighbourhoods_gdf, traffic_cameras_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of traffic cameras as a new column to the neighborhoods GeoDataFrame
neighbourhoods_gdf['TCamera_Count'] = neighbourhoods_gdf['NeighbourhoodName'].map(traffic_cameras_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head()

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534,1
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876,1
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281,12


In [11]:
import pandas as pd

# Path to the CSV file
cenus_file_path = os.path.join(folder_path, 'CensusProfile_2021_PopAndIncome_Reformatted.csv')

# Read the CSV file into a DataFrame
census_df = pd.read_csv(cenus_file_path)

# Display the first few rows of the GeoDataFrame
census_df.head()

,Geography,TotalPopulation,MedianTotalIncome,AverageTotalIncome,Neighbourhood
0,Neighbourhood - 1 (West Humber-Clairville) 00...,33300,33600,40560,West Humber-Clairville
1,Neighbourhood - 2 (Mount Olive-Silverstone-Ja...,31345,29600,35000,Mount Olive-Silverstone-Jamestown
2,Neighbourhood - 3 (Thistletown-Beaumond Heigh...,9850,32800,41520,Thistletown-Beaumond Heights
3,Neighbourhood - 4 (Rexdale-Kipling) 00000 ( ...,10375,33600,42040,Rexdale-Kipling
4,Neighbourhood - 5 (Elms-Old Rexdale) 00000 ( ...,9355,34400,41240,Elms-Old Rexdale


In [12]:
neighbourhoods_gdf.head()

,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534,1
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876,1
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281,12


In [13]:
# Merge the DataFrame with the neighborhoods GeoDataFrame based on the "NeighbourhoodName" column
neighbourhoods_gdf = pd.merge(neighbourhoods_gdf, census_df, left_on='NeighbourhoodName', right_on='Neighbourhood', how='left')

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head(2)

,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,Geography,TotalPopulation,MedianTotalIncome,AverageTotalIncome,Neighbourhood
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2,Neighbourhood - 174 (South Eglinton-Davisvill...,22735,52400,69000,South Eglinton-Davisville
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2,Neighbourhood - 173 (North Toronto) 00000 ( ...,15885,46000,58100,North Toronto


In [14]:
# Drop columns
neighbourhoods_gdf.drop(['Geography', 'Neighbourhood'], axis=1, inplace=True)

# Display updated neighborhoods GeoDataFrame
neighbourhoods_gdf.head(2)

,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2,22735,52400,69000
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2,15885,46000,58100


In [15]:
# Load the CSV file
Buildings_file = os.path.join(folder_path, "TI_Neighbourhoods_BuildingDissolved.csv")
Buildings_data = pd.read_csv(Buildings_file)

# Merge the CSV data with neighbourhoods_gdf based on the NeighbourhoodName field
neighbourhoods_gdf = pd.merge(neighbourhoods_gdf, Buildings_data, left_on="NeighbourhoodName", right_on="AREA_NA7")

In [16]:
# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,OBJECTID,AREA_NA7,AREA,BuildingCoveragePERCENTAGE
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2,22735,52400,69000,123,South Eglinton-Davisville,3.789351e+05,29.087555
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2,15885,46000,58100,103,North Toronto,1.741970e+05,31.403497
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534,1,12380,38000,55850,35,Dovercourt Village,6.886593e+05,33.231409
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,71,Junction-Wallace Emerson,1.016517e+06,33.167140
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,153,Yonge-Bay Corridor,7.406525e+05,48.028587


In [17]:
# Specify columns to drop
columns_to_drop = ['OBJECTID', 'AREA_NA7', 'AREA']

# Drop the specified columns from neighbourhoods_gdf
neighbourhoods_gdf = neighbourhoods_gdf.drop(columns=columns_to_drop)

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2,15885,46000,58100,31.403497
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534,1,12380,38000,55850,33.231409
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587


In [18]:
# Load the CSV file
ParkOSNA_file = os.path.join(folder_path, "TI_Neighbourhoods_ParksOpenSpaceNaturalArea.csv")
ParkOSNA_data = pd.read_csv(ParkOSNA_file)

# Merge the CSV data with neighbourhoods_gdf based on the NeighbourhoodName field
neighbourhoods_gdf = pd.merge(neighbourhoods_gdf, ParkOSNA_data, left_on="NeighbourhoodName", right_on="AREA_NA7")

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE,OBJECTID,AREA_NA7,AREA,ParksOSNAPERCENTAGE
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555,123,South Eglinton-Davisville,60207.464870,6.375824
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2,15885,46000,58100,31.403497,103,North Toronto,4556.163857,1.133288
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534,1,12380,38000,55850,33.231409,35,Dovercourt Village,118362.095300,7.875048
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140,71,Junction-Wallace Emerson,151633.301600,6.821523
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587,153,Yonge-Bay Corridor,71397.600170,6.382056


In [19]:
# Specify columns to drop
columns_to_drop = ['OBJECTID', 'AREA_NA7', 'AREA']

# Drop the specified columns from neighbourhoods_gdf
neighbourhoods_gdf = neighbourhoods_gdf.drop(columns=columns_to_drop)

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE,ParksOSNAPERCENTAGE
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555,6.375824
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2,15885,46000,58100,31.403497,1.133288
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534,1,12380,38000,55850,33.231409,7.875048
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140,6.821523
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587,6.382056


In [20]:
# Subtract "ParksOSNAPERCENTAGE" from 100 to get the "BuiltUpPERCENTAGE"
neighbourhoods_gdf['BuiltUpPERCENTAGE'] = 100 - neighbourhoods_gdf['ParksOSNAPERCENTAGE']

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE,ParksOSNAPERCENTAGE,BuiltUpPERCENTAGE
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555,6.375824,93.624176
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2,15885,46000,58100,31.403497,1.133288,98.866712
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534,1,12380,38000,55850,33.231409,7.875048,92.124952
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140,6.821523,93.178477
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587,6.382056,93.617944


In [21]:
# Load avg home prices CSV file
AvgHomePrice = os.path.join(folder_path, "AverageHomePrices_2023.csv")
AvgHomePrice_data = pd.read_csv(AvgHomePrice)

# Merge the CSV data with neighbourhoods_gdf based on the NeighbourhoodName field
neighbourhoods_gdf = pd.merge(neighbourhoods_gdf, AvgHomePrice_data, left_on="NeighbourhoodName", right_on="Neighbourhood")

In [22]:
# Specify columns to drop
columns_to_drop = ['UID', 'Neighbourhood', 'TREB_Code']

# Drop the specified columns from neighbourhoods_gdf
neighbourhoods_gdf = neighbourhoods_gdf.drop(columns=columns_to_drop)

# Display the updated neighbourhoods_gdf
neighbourhoods_gdf.head()

,geometry,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count,TotalPopulation,MedianTotalIncome,AverageTotalIncome,BuildingCoveragePERCENTAGE,ParksOSNAPERCENTAGE,BuiltUpPERCENTAGE,AveragePrice
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",South Eglinton-Davisville,649,1185,2,22735,52400,69000,29.087555,6.375824,93.624176,1035636
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",North Toronto,286,851,2,15885,46000,58100,31.403497,1.133288,98.866712,1035636
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",Dovercourt Village,1170,534,1,12380,38000,55850,33.231409,7.875048,92.124952,1266076
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",Junction-Wallace Emerson,1663,876,1,23180,41200,54350,33.167140,6.821523,93.178477,1266076
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",Yonge-Bay Corridor,1139,20281,12,12645,44000,68200,48.028587,6.382056,93.617944,880955


In [23]:
# Check for null values in each column
null_counts = neighbourhoods_gdf.isnull().sum()

# Display null counts for each column
print("Null counts for each column:")
print(null_counts)

Null counts for each column:
geometry                      0
NeighbourhoodName             0
Poles_Count                   0
POI_Count                     0
TCamera_Count                 0
TotalPopulation               0
MedianTotalIncome             0
AverageTotalIncome            0
BuildingCoveragePERCENTAGE    0
ParksOSNAPERCENTAGE           0
BuiltUpPERCENTAGE             0
AveragePrice                  0
dtype: int64


In [24]:
# Export the DataFrame to a CSV file
neighbourhoods_gdf.to_csv(os.path.join(folder_path, 'Neighbourhood_Cleaned.csv'), index=False)

In [25]:
# Load the ClimateData_Hourly.csv file
ClimateData_file = os.path.join(folder_path, "ClimateData_Hourly.csv")
ClimateData_data = pd.read_csv(ClimateData_file)

# List all fields in the file
fields_in_file = ClimateData_data.columns.tolist()

# Display the list of fields
print(fields_in_file)

['x', 'y', 'UTC_DATE', 'UTC_MONTH', 'STATION_PRESSURE', 'WEATHER_ENG_DESC', 'DATE_TIME_LINK', 'LOCAL_YEAR', 'LOCAL_MONTH', 'WINDCHILL_FLAG', 'UTC_YEAR', 'LOCAL_DAY', 'VISIBILITY', 'VISIBILITY_FLAG', 'PRECIP_AMOUNT', 'RELATIVE_HUMIDITY', 'PROVINCE_CODE', 'TEMP_FLAG', 'LOCAL_HOUR', 'WIND_DIRECTION_FLAG', 'STATION_PRESSURE_FLAG', 'PRECIP_AMOUNT_FLAG', 'STATION_NAME', 'CLIMATE_IDENTIFIER', 'UTC_DAY', 'WIND_SPEED_FLAG', 'WIND_DIRECTION', 'DEW_POINT_TEMP_FLAG', 'WEATHER_FRE_DESC', 'HUMIDEX', 'RELATIVE_HUMIDITY_FLAG', 'LOCAL_DATE', 'WINDCHILL', 'TEMP', 'ID', 'WIND_SPEED', 'DEW_POINT_TEMP', 'HUMIDEX_FLAG']


C:\Users\rdtruong\AppData\Local\Temp\ipykernel_7116\688971924.py:3: DtypeWarning: Columns (17,19,20,21,25,27,30) have mixed types. Specify dtype option on import or set low_memory=False.
  ClimateData_data = pd.read_csv(ClimateData_file)


In [26]:
# Keep only the 'UTC_DATE_FORMATTED' and 'TEMP' fields
ClimateData_data = ClimateData_data[['DATE_TIME_LINK', 'TEMP']]

# Display the updated DataFrame
ClimateData_data.head()

,DATE_TIME_LINK,TEMP
0,2013/12/31 0,-9.1
1,2013/12/31 1,-8.9
2,2013/12/31 10,-5.9
3,2013/12/31 11,-6.0
4,2013/12/31 12,-6.8


In [27]:
# Convert 'TEMP' column to numeric values, coercing errors to NaN
ClimateData_data['TEMP'] = pd.to_numeric(ClimateData_data['TEMP'], errors='coerce')

# Define a function to categorize temperature
def categorize_temperature(temp):
    if pd.isnull(temp) or temp == "":
        return None  # If temperature is blank or null, return None
    elif temp <= -35:
        return "Extreme Cold"
    elif -35 < temp <= -20:
        return "Very Cold"
    elif -20 < temp <= -10:
        return "Cold"
    elif -10 < temp <= -5:
        return "Chilly"
    elif -5 < temp <= 0:
        return "Cool"
    elif 0 < temp <= 10:
        return "Mild"
    elif 10 < temp <= 20:
        return "Warm"
    elif 20 < temp <= 29:
        return "Hot"
    else:
        return "Very Hot"

# Create the TEMPCategory field using the categorize_temperature function
ClimateData_data['TEMPCategory'] = ClimateData_data['TEMP'].apply(categorize_temperature)

In [28]:
# Display the first few rows to verify
print(ClimateData_data.head(10))

  DATE_TIME_LINK  TEMP TEMPCategory
0   2013/12/31 0  -9.1       Chilly
1   2013/12/31 1  -8.9       Chilly
2  2013/12/31 10  -5.9       Chilly
3  2013/12/31 11  -6.0       Chilly
4  2013/12/31 12  -6.8       Chilly
5  2013/12/31 13  -6.3       Chilly
6  2013/12/31 14  -7.3       Chilly
7  2013/12/31 15  -7.6       Chilly
8  2013/12/31 16  -7.9       Chilly
9  2013/12/31 17  -8.5       Chilly


In [29]:
# Load the CSV file
MajorCrime_file = os.path.join(folder_path, "MajorCrimeIndicators.csv")
MajorCrime_data = pd.read_csv(MajorCrime_file)

# List all fields in the file
fields_in_file = MajorCrime_data.columns.tolist()

# Display the list of fields
print(fields_in_file)

['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE', 'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW', 'REPORT_HOUR', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE', 'UCR_EXT', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158', 'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84']


In [30]:
# Find the number of records in the filtered data
num_records = len(MajorCrime_data)

# Display the number of records
print("Number of records:", num_records)

Number of records: 372899


In [31]:
# List unique attributes in the OCC_YEAR column
unique_years = MajorCrime_data['OCC_YEAR'].unique()

# Display the unique attributes
print("Unique years:", unique_years)

Unique years: [2023. 2022. 2021. 2020. 2019. 2018. 2017. 2016. 2015. 2014. 2013. 2012.
 2011. 2010. 2009. 2008. 2007. 2006. 2005. 2004. 2003. 2002. 2001. 2000.
   nan]


In [32]:
# Filter the data where OCC_YEAR is between 2014 and 2023 (inclusive)
MajorCrime_data_filtered = MajorCrime_data[(MajorCrime_data['OCC_YEAR'] >= 2014) & (MajorCrime_data['OCC_YEAR'] <= 2023)]

# Display the filtered data
MajorCrime_data_filtered.head()

,X,Y,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-8844392.411,5415470.133,323507,GO-2023916,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,1610,100,Robbery With Weapon,Robbery,92,Corso Italia-Davenport,92,Corso Italia-Davenport (92),-79.450529,43.676205
1,-8835256.191,5411249.380,323508,GO-2023748,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,1610,220,Robbery - Other,Robbery,166,St Lawrence-East Bayfront-The Islands,77,Waterfront Communities-The Island (77),-79.368457,43.648776
2,-8816797.331,5436899.712,323509,GO-20232814,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,1430,100,Assault,Assault,144,Morningside Heights,131,Rouge (131),-79.202638,43.815274
3,-8851797.747,5403889.115,323510,GO-20231708,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,1420,100,Assault With Weapon,Assault,18,New Toronto,18,New Toronto (18),-79.517052,43.600915
4,-8827699.146,5422167.524,323511,GO-20236562,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,2135,210,Theft Of Motor Vehicle,Auto Theft,43,Victoria Village,43,Victoria Village (43),-79.300571,43.719703


In [33]:
# Find the number of records in the filtered data
num_records = len(MajorCrime_data_filtered)

# Display the number of records
print("Number of records:", num_records)

Number of records: 371364


In [34]:
# List unique attributes in the OCC_YEAR column
unique_years = MajorCrime_data_filtered['OCC_YEAR'].unique()

# Display the unique attributes
print("Unique years:", unique_years)

Unique years: [2023. 2022. 2021. 2020. 2019. 2018. 2017. 2016. 2015. 2014.]


In [35]:
# Filter out rows where either LONG_WGS84 or LAT_WGS84 equals 0
MajorCrime_data_filtered = MajorCrime_data_filtered[(MajorCrime_data_filtered['LONG_WGS84'] != 0) & (MajorCrime_data_filtered['LAT_WGS84'] != 0)]

# Find the number of records in the filtered data
num_records = len(MajorCrime_data_filtered)

# Display the number of records
print("Number of records:", num_records)

Number of records: 365686


In [36]:
# Define a function to convert month names to month numbers
def month_to_num(month):
    month_dict = {
        'January': 1,
        'February': 2,
        'March': 3,
        'April': 4,
        'May': 5,
        'June': 6,
        'July': 7,
        'August': 8,
        'September': 9,
        'October': 10,
        'November': 11,
        'December': 12
    }
    return month_dict.get(month)

# Apply the function to create the new field
MajorCrime_data_filtered['MonthNum'] = MajorCrime_data_filtered['OCC_MONTH'].apply(month_to_num)

In [37]:
# Display the DataFrame with the new field
MajorCrime_data_filtered

,X,Y,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,...,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84,MonthNum
0,-8844392.411,5415470.133,323507,GO-2023916,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,100,Robbery With Weapon,Robbery,92,Corso Italia-Davenport,92,Corso Italia-Davenport (92),-79.450529,43.676205,1
1,-8835256.191,5411249.380,323508,GO-2023748,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,220,Robbery - Other,Robbery,166,St Lawrence-East Bayfront-The Islands,77,Waterfront Communities-The Island (77),-79.368457,43.648776,1
2,-8816797.331,5436899.712,323509,GO-20232814,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,100,Assault,Assault,144,Morningside Heights,131,Rouge (131),-79.202638,43.815274,1
3,-8851797.747,5403889.115,323510,GO-20231708,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,100,Assault With Weapon,Assault,18,New Toronto,18,New Toronto (18),-79.517052,43.600915,1
4,-8827699.146,5422167.524,323511,GO-20236562,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,210,Theft Of Motor Vehicle,Auto Theft,43,Victoria Village,43,Victoria Village (43),-79.300571,43.719703,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371359,-8837684.969,5411106.705,34285,GO-2015133590,2015/01/23 05:00:00+00,2014/12/31 05:00:00+00,2015,January,23,23,...,200,B&E,Break and Enter,164,Wellington Place,77,Waterfront Communities-The Island (77),-79.390275,43.647849,12
371360,-8821923.792,5428650.252,37333,GO-2015373141,2015/03/04 05:00:00+00,2014/12/31 05:00:00+00,2015,March,4,63,...,100,Assault With Weapon,Assault,156,Bendale-Glen Andrew,127,Bendale (127),-79.248690,43.761776,12
371361,-8821923.792,5428650.252,37349,GO-2015373141,2015/03/04 05:00:00+00,2014/12/31 05:00:00+00,2015,March,4,63,...,100,Assault,Assault,156,Bendale-Glen Andrew,127,Bendale (127),-79.248690,43.761776,12
371362,-8830522.003,5414727.719,45652,GO-2015954354,2015/06/07 04:00:00+00,2014/12/31 05:00:00+00,2015,June,7,158,...,100,Assault,Assault,65,Greenwood-Coxwell,65,Greenwood-Coxwell (65),-79.325929,43.671381,12


In [38]:
# Convert OCC_YEAR, OCC_MONTH, and OCC_DAY to integers
MajorCrime_data_filtered['OCC_YEAR'] = MajorCrime_data_filtered['OCC_YEAR'].astype(int)
MajorCrime_data_filtered['OCC_DAY'] = MajorCrime_data_filtered['OCC_DAY'].astype(int)
MajorCrime_data_filtered['OCC_DOY'] = MajorCrime_data_filtered['OCC_DOY'].astype(int)

# Define a function to convert month names to month numbers
def month_to_num(month):
    month_dict = {
        'January': 1,
        'February': 2,
        'March': 3,
        'April': 4,
        'May': 5,
        'June': 6,
        'July': 7,
        'August': 8,
        'September': 9,
        'October': 10,
        'November': 11,
        'December': 12
    }
    return month_dict.get(month)

# Define a function to create the date-time link
def create_date_time_link(year, month, day, hour):
    return f"{year}/{month}/{day} {hour}"

# Apply the function to create the new field
MajorCrime_data_filtered['MonthNum'] = MajorCrime_data_filtered['OCC_MONTH'].apply(month_to_num)
MajorCrime_data_filtered['DATE_TIME_LINK'] = MajorCrime_data_filtered.apply(lambda row: create_date_time_link(row['OCC_YEAR'], row['MonthNum'], row['OCC_DAY'], row['OCC_HOUR']), axis=1)

In [39]:
# Display the DataFrame with the new field
MajorCrime_data_filtered

,X,Y,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,...,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84,MonthNum,DATE_TIME_LINK
0,-8844392.411,5415470.133,323507,GO-2023916,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,Robbery With Weapon,Robbery,92,Corso Italia-Davenport,92,Corso Italia-Davenport (92),-79.450529,43.676205,1,2023/1/1 2
1,-8835256.191,5411249.380,323508,GO-2023748,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,Robbery - Other,Robbery,166,St Lawrence-East Bayfront-The Islands,77,Waterfront Communities-The Island (77),-79.368457,43.648776,1,2023/1/1 1
2,-8816797.331,5436899.712,323509,GO-20232814,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,Assault,Assault,144,Morningside Heights,131,Rouge (131),-79.202638,43.815274,1,2023/1/1 9
3,-8851797.747,5403889.115,323510,GO-20231708,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,Assault With Weapon,Assault,18,New Toronto,18,New Toronto (18),-79.517052,43.600915,1,2023/1/1 3
4,-8827699.146,5422167.524,323511,GO-20236562,2023/01/01 05:00:00+00,2023/01/01 05:00:00+00,2023,January,1,1,...,Theft Of Motor Vehicle,Auto Theft,43,Victoria Village,43,Victoria Village (43),-79.300571,43.719703,1,2023/1/1 21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371359,-8837684.969,5411106.705,34285,GO-2015133590,2015/01/23 05:00:00+00,2014/12/31 05:00:00+00,2015,January,23,23,...,B&E,Break and Enter,164,Wellington Place,77,Waterfront Communities-The Island (77),-79.390275,43.647849,12,2014/12/31 16
371360,-8821923.792,5428650.252,37333,GO-2015373141,2015/03/04 05:00:00+00,2014/12/31 05:00:00+00,2015,March,4,63,...,Assault With Weapon,Assault,156,Bendale-Glen Andrew,127,Bendale (127),-79.248690,43.761776,12,2014/12/31 12
371361,-8821923.792,5428650.252,37349,GO-2015373141,2015/03/04 05:00:00+00,2014/12/31 05:00:00+00,2015,March,4,63,...,Assault,Assault,156,Bendale-Glen Andrew,127,Bendale (127),-79.248690,43.761776,12,2014/12/31 12
371362,-8830522.003,5414727.719,45652,GO-2015954354,2015/06/07 04:00:00+00,2014/12/31 05:00:00+00,2015,June,7,158,...,Assault,Assault,65,Greenwood-Coxwell,65,Greenwood-Coxwell (65),-79.325929,43.671381,12,2014/12/31 22


In [40]:
# List of fields to keep
fields_to_keep = ['OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW', 'OCC_HOUR', 'LOCATION_TYPE', 'PREMISES_TYPE', 'OFFENCE', 'MCI_CATEGORY','NEIGHBOURHOOD_158', 'LONG_WGS84', 'LAT_WGS84', 'DATE_TIME_LINK']

# Keep only the desired fields using direct column indexing
MajorCrime_data_filtered = MajorCrime_data_filtered[fields_to_keep]

In [41]:
# Display the updated DataFrame
MajorCrime_data_filtered

,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_DOY,OCC_DOW,OCC_HOUR,LOCATION_TYPE,PREMISES_TYPE,OFFENCE,MCI_CATEGORY,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,DATE_TIME_LINK
0,2023,January,1,1,Sunday,2,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery With Weapon,Robbery,Corso Italia-Davenport,-79.450529,43.676205,2023/1/1 2
1,2023,January,1,1,Sunday,1,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery - Other,Robbery,St Lawrence-East Bayfront-The Islands,-79.368457,43.648776,2023/1/1 1
2,2023,January,1,1,Sunday,9,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Morningside Heights,-79.202638,43.815274,2023/1/1 9
3,2023,January,1,1,Sunday,3,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Assault With Weapon,Assault,New Toronto,-79.517052,43.600915,2023/1/1 3
4,2023,January,1,1,Sunday,21,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Theft Of Motor Vehicle,Auto Theft,Victoria Village,-79.300571,43.719703,2023/1/1 21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371359,2014,December,31,365,Wednesday,16,"Apartment (Rooming House, Condo)",Apartment,B&E,Break and Enter,Wellington Place,-79.390275,43.647849,2014/12/31 16
371360,2014,December,31,365,Wednesday,12,"Single Home, House (Attach Garage, Cottage, Mo...",House,Assault With Weapon,Assault,Bendale-Glen Andrew,-79.248690,43.761776,2014/12/31 12
371361,2014,December,31,365,Wednesday,12,"Single Home, House (Attach Garage, Cottage, Mo...",House,Assault,Assault,Bendale-Glen Andrew,-79.248690,43.761776,2014/12/31 12
371362,2014,December,31,365,Wednesday,22,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Greenwood-Coxwell,-79.325929,43.671381,2014/12/31 22


In [42]:
# Define a function to categorize hours
def categorize_time(hour):
    if 6 <= hour <= 11:
        return 'Morning'
    elif 12 <= hour <= 17:
        return 'Afternoon'
    elif 18 <= hour <= 23:
        return 'Evening'
    else:
        return 'Night'

# Apply the function to create the new field
MajorCrime_data_filtered.loc[:,'TIMECategory'] = MajorCrime_data_filtered['OCC_HOUR'].apply(categorize_time)

C:\Users\rdtruong\AppData\Local\Temp\ipykernel_7116\3366365470.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MajorCrime_data_filtered.loc[:,'TIMECategory'] = MajorCrime_data_filtered['OCC_HOUR'].apply(categorize_time)


In [43]:
# Display the DataFrame with the new field
MajorCrime_data_filtered.head(10)

,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_DOY,OCC_DOW,OCC_HOUR,LOCATION_TYPE,PREMISES_TYPE,OFFENCE,MCI_CATEGORY,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,DATE_TIME_LINK,TIMECategory
0,2023,January,1,1,Sunday,2,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery With Weapon,Robbery,Corso Italia-Davenport,-79.450529,43.676205,2023/1/1 2,Night
1,2023,January,1,1,Sunday,1,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery - Other,Robbery,St Lawrence-East Bayfront-The Islands,-79.368457,43.648776,2023/1/1 1,Night
2,2023,January,1,1,Sunday,9,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Morningside Heights,-79.202638,43.815274,2023/1/1 9,Morning
3,2023,January,1,1,Sunday,3,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Assault With Weapon,Assault,New Toronto,-79.517052,43.600915,2023/1/1 3,Night
4,2023,January,1,1,Sunday,21,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Theft Of Motor Vehicle,Auto Theft,Victoria Village,-79.300571,43.719703,2023/1/1 21,Evening
5,2023,January,1,1,Sunday,21,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Theft Of Motor Vehicle,Auto Theft,Woodbine Corridor,-79.309287,43.677265,2023/1/1 21,Evening
6,2023,January,1,1,Sunday,3,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Assault,Assault,Annex,-79.402690,43.666904,2023/1/1 3,Night
7,2023,January,1,1,Sunday,6,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,North St.James Town,-79.371458,43.671891,2023/1/1 6,Morning
8,2023,January,1,1,Sunday,7,"Single Home, House (Attach Garage, Cottage, Mo...",House,Assault,Assault,Morningside Heights,-79.185494,43.806636,2023/1/1 7,Morning
9,2023,January,1,1,Sunday,7,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Church-Wellesley,-79.378334,43.666282,2023/1/1 7,Morning


In [44]:
# Merge the climate data with the crime data based on the date columns
MajorCrime_data_filtered = pd.merge(MajorCrime_data_filtered, ClimateData_data, left_on='DATE_TIME_LINK', right_on='DATE_TIME_LINK', how='left')

In [45]:
# Display the merged data
MajorCrime_data_filtered.head()

,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_DOY,OCC_DOW,OCC_HOUR,LOCATION_TYPE,PREMISES_TYPE,OFFENCE,MCI_CATEGORY,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,DATE_TIME_LINK,TIMECategory,TEMP,TEMPCategory
0,2023,January,1,1,Sunday,2,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery With Weapon,Robbery,Corso Italia-Davenport,-79.450529,43.676205,2023/1/1 2,Night,4.7,Mild
1,2023,January,1,1,Sunday,1,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Robbery - Other,Robbery,St Lawrence-East Bayfront-The Islands,-79.368457,43.648776,2023/1/1 1,Night,4.5,Mild
2,2023,January,1,1,Sunday,9,"Apartment (Rooming House, Condo)",Apartment,Assault,Assault,Morningside Heights,-79.202638,43.815274,2023/1/1 9,Morning,4.4,Mild
3,2023,January,1,1,Sunday,3,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Assault With Weapon,Assault,New Toronto,-79.517052,43.600915,2023/1/1 3,Night,4.9,Mild
4,2023,January,1,1,Sunday,21,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,Theft Of Motor Vehicle,Auto Theft,Victoria Village,-79.300571,43.719703,2023/1/1 21,Evening,4.1,Mild


In [46]:
# Check for null values in each column
null_counts = MajorCrime_data_filtered.isnull().sum()

# Display null counts for each column
print("Null counts for each column:")
print(null_counts)

Null counts for each column:
OCC_YEAR                0
OCC_MONTH               0
OCC_DAY                 0
OCC_DOY                 0
OCC_DOW                 0
OCC_HOUR                0
LOCATION_TYPE           0
PREMISES_TYPE           0
OFFENCE                 0
MCI_CATEGORY            0
NEIGHBOURHOOD_158       0
LONG_WGS84              0
LAT_WGS84               0
DATE_TIME_LINK          0
TIMECategory            0
TEMP                 1329
TEMPCategory         1329
dtype: int64


In [47]:
# Display information about the DataFrame
print("Information about MajorCrime_data_filtered:")
MajorCrime_data_filtered.info()

Information about MajorCrime_data_filtered:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365686 entries, 0 to 365685
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   OCC_YEAR           365686 non-null  int32  
 1   OCC_MONTH          365686 non-null  object 
 2   OCC_DAY            365686 non-null  int32  
 3   OCC_DOY            365686 non-null  int32  
 4   OCC_DOW            365686 non-null  object 
 5   OCC_HOUR           365686 non-null  int64  
 6   LOCATION_TYPE      365686 non-null  object 
 7   PREMISES_TYPE      365686 non-null  object 
 8   OFFENCE            365686 non-null  object 
 9   MCI_CATEGORY       365686 non-null  object 
 10  NEIGHBOURHOOD_158  365686 non-null  object 
 11  LONG_WGS84         365686 non-null  float64
 12  LAT_WGS84          365686 non-null  float64
 13  DATE_TIME_LINK     365686 non-null  object 
 14  TIMECategory       365686 non-null  object 
 15  TEMP   

In [48]:
# Export the DataFrame to a CSV file
MajorCrime_data_filtered.to_csv(os.path.join(folder_path, 'MCI_Cleaned.csv'), index=False)